### HOUSE PRICING 1 : EXTRACTION 

Import the necessary libraries

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import boto3
import numpy as np
from geopy.geocoders import Nominatim

Set the parameters to start our AWS client

In [3]:
params = {
    "region": "region",
    "bucket": "bucket", 
    "key":"key",
    'folder' : 'folder',
    "secretkey":"secretkey"}

'params = {\n    "region": "region",\n    "bucket": "bucket", \n    "key":"key",\n\n    "secretkey":"secretkey"}'

Start the s3 client, we will use it later

In [4]:
s3 = boto3.client(
        's3', 
        region_name = params['region'],
        aws_access_key_id = params['key'],
        aws_secret_access_key = params['secretkey']
    )

Create a empty dataframe

In [5]:
real_estate = pd.DataFrame(columns=['Localizacion' ,'Precio', 'Cuartos', 'Baños', 'Cocheras', 
                                    'Area_total', 'Longitud', 'Latitud'])

I define a function to avoid errors with null values

In [6]:
def get_item_by_class(result_reg,localizador, clase):
    '''
    This funtions gets the text from determinated instances using the class
    '''
    lista = list()
    for i in result_reg:
        try:
            item = i.find(localizador, {'class': clase}).get_text()
            lista.append(item)
        except:
            lista.append('Null')
    return lista

I define 2 functions to find the longitude and latitude

In [8]:
geolocator = Nominatim(user_agent="MyApp")

def ext_longitud(direct):
    try:
        location = geolocator.geocode(direct)
        longitud = location.longitude
    except:
        longitud = None
        
    return longitud

    
def ext_latitud(direct):
    try:
        location = geolocator.geocode(direct)
        latitud = location.latitude
    except:
        latitud = None
        
    return latitud

Scrape the information. I use 50 pages per state and every page give us 23 new rows

In [9]:
for i in range(51):
    print('iteracion: ', i)
    website = requests.get('https://www.vivanuncios.com.mx/s-venta-inmuebles/estado-de-mexico/page-'+str(i)+'/v1c1097l1014p'+str(i))
    soup = BeautifulSoup(website.content, 'html.parser')

    result_reg = soup.find_all('div', {'class':'tileV2 REAdTileV2 regular listView'})

    Localizacion = get_item_by_class(result_reg,'div', 'tile-location one-liner')
                
    Cocheras = get_item_by_class(result_reg,'div', 'chiplets-inline-block car-parking')
                
    Cuartos = get_item_by_class(result_reg,'div', 'chiplets-inline-block re-bedroom')
                
    Baños = get_item_by_class(result_reg,'div', 'chiplets-inline-block re-bathroom')
                
    Area_total = get_item_by_class(result_reg,'div', 'chiplets-inline-block surface-area')
                
    Precio = get_item_by_class(result_reg,'span', 'ad-price')
    Precio = [i.strip() for i in Precio]
    
    Longitud = [ext_longitud(x + ', Mexico') for x in Localizacion ]
    
    Latitud = [ext_latitud(x + ', Mexico') for x in Localizacion ]


    for k in range(len(Localizacion)):
        real_estate = real_estate.append({'Localizacion' : Localizacion[k],  'Precio': Precio[k], 
                                            'Cuartos': Cuartos[k], 'Baños' : Baños[k], 
                                            'Cocheras': Cocheras[k], 'Area_total': Area_total[k],
                                            'Longitud': Longitud[k], 'Latitud': Latitud[k]},
                                        ignore_index=True)

iteracion:  0
iteracion:  1
iteracion:  2
iteracion:  3
iteracion:  4
iteracion:  5
iteracion:  6
iteracion:  7
iteracion:  8
iteracion:  9
iteracion:  10
iteracion:  11
iteracion:  12
iteracion:  13
iteracion:  14
iteracion:  15
iteracion:  16
iteracion:  17
iteracion:  18
iteracion:  19
iteracion:  20
iteracion:  21
iteracion:  22
iteracion:  23
iteracion:  24
iteracion:  25
iteracion:  26
iteracion:  27
iteracion:  28
iteracion:  29
iteracion:  30
iteracion:  31
iteracion:  32
iteracion:  33
iteracion:  34
iteracion:  35
iteracion:  36
iteracion:  37
iteracion:  38
iteracion:  39
iteracion:  40
iteracion:  41
iteracion:  42
iteracion:  43
iteracion:  44
iteracion:  45
iteracion:  46
iteracion:  47
iteracion:  48
iteracion:  49
iteracion:  50


In [10]:
real_estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Localizacion  1148 non-null   object 
 1   Precio        1148 non-null   object 
 2   Cuartos       1148 non-null   object 
 3   Baños         1148 non-null   object 
 4   Cocheras      1148 non-null   object 
 5   Area_total    1148 non-null   object 
 6   Longitud      973 non-null    float64
 7   Latitud       968 non-null    object 
dtypes: float64(1), object(7)
memory usage: 71.9+ KB


In [11]:
real_estate.head(10)

,Localizacion,Precio,Cuartos,Baños,Cocheras,Area_total,Longitud,Latitud
0,La Loma,"$994,700",4,2,Null,Null,-108.130203,25.405243
1,San Jose,"$3,280,000",3,2,Null,116 m²,-105.475657,35.39724
2,Lomas de Bellavista,"$11,500,000",3,4+,3,254 m²,-102.222543,17.973581
3,Ocho Cedros,"$1,200,000",6+,4+,2,Null,NaN,NaN
4,Heroes de la Independencia,"$1,200,000",5,3,1,120 m²,-115.891498,31.611398
5,Ecatepec de Morelos,"$1,150,000",3,2,1,30 m²,-99.048728,19.599408
6,Los Heroes Coacalco,"$1,150,000",2,1,1,Null,NaN,NaN
7,El Paraiso,"$1,445,082",3,4+,2,129 m²,-93.551530,16.246132
8,Ex-Hacienda Santa Ines,"$530,000",1,1,Null,60 m²,NaN,NaN
9,Estado de Mexico,"$3,100,000",4,3,Null,215 m²,-99.689972,19.483945


I save the dataframe to csv

In [12]:
real_estate.to_csv('real_state.csv', index= False)

Upload the file to s3 in the bucket and folder defined in the parameters

In [14]:
s3.upload_file('real_state.csv', params['bucket'], params['folder'] +'real_state.csv')       
